### 2.1 下载ChatGLM3模型

In [1]:
import os
import moxing as mox

#拷贝chatglm3模型到chatglm3-6b目录下
work_dir = '/home/ma-user/work'
obs_path = 'obs://dtse-models/tar-models/chatglm3-6b.tar'
ma_path = os.path.join(work_dir, 'chatglm3-6b.tar')
mox.file.copy(obs_path, ma_path)

mox.file.copy_parallel('obs://modelarts-labs-bj4-v2/course/ModelBox/frpc_linux_amd64', '/home/ma-user/work/frpc_linux_amd64')

INFO:root:Using MoXing-v2.1.0.5d9c87c8-5d9c87c8
INFO:root:Using OBS-Python-SDK-3.20.9.1


解压chatglm3模型压缩包

In [2]:
os.chdir(work_dir)
!pwd
!tar -xvf chatglm3-6b.tar

/home/ma-user/work
chatglm3-6b/
chatglm3-6b/.gitattributes
chatglm3-6b/MODEL_LICENSE
chatglm3-6b/README.md
chatglm3-6b/config.json
chatglm3-6b/configuration_chatglm.py
chatglm3-6b/modeling_chatglm.py
chatglm3-6b/pytorch_model-00001-of-00007.bin
chatglm3-6b/pytorch_model-00002-of-00007.bin
chatglm3-6b/pytorch_model-00003-of-00007.bin
chatglm3-6b/pytorch_model-00004-of-00007.bin
chatglm3-6b/pytorch_model-00005-of-00007.bin
chatglm3-6b/pytorch_model-00006-of-00007.bin
chatglm3-6b/pytorch_model-00007-of-00007.bin
chatglm3-6b/pytorch_model.bin.index.json
chatglm3-6b/quantization.py
chatglm3-6b/tokenization_chatglm.py
chatglm3-6b/tokenizer.model
chatglm3-6b/tokenizer_config.json


### 2.2 配置环境

本案例依赖Python3.10.10及以上环境，因此我们首先创建虚拟环境：

In [3]:
!/home/ma-user/anaconda3/bin/conda create -n python-3.10.10 python=3.10.10 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.10.10/bin/pip install ipykernel

/home/ma-user/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /home/ma-user/anaconda3/envs/python-3.10.10

  added / updated specs:
    - python=3.10.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           3 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    _openmp_mutex-5.1          |            1_gnu          21 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    bzip2-1.0.8                |       h7b6447c_0          78 KB  https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
    ca-certif

In [6]:
import json
import os

data = {
   "display_name": "python-3.10.10",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.10.10/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/anaconda3/envs/PyTorch-1.8/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.10.10/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

### 2.3 安装依赖库

In [1]:
!pip install transformers==4.30.2
!pip install sentencepiece==0.1.99
!pip install torch==2.0.1
!pip install gradio==3.39 mdtex2html -i https://pypi.tuna.tsinghua.edu.cn/simple --trusted-host pypi.tuna.tsinghua.edu.cn
!cp /home/ma-user/work/frpc_linux_amd64 /home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2
!chmod +x /home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.5 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 105.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 11.0 MB/s eta 0:00:00
     

In [2]:
!pip install transformers==4.30.2
!pip install sentencepiece==0.1.99
!pip install torch==2.0.1
!pip install langchain==0.0.329 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.4/613.4 kB 9.4 MB/s eta 0:00:00a 0:00:01


### 模型测试
在课程2/3的基础上实现一个于基ChatGLM3+LangChain的聊天应用，需要有Gradio界面


In [3]:
import os
from typing import Optional, List
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from transformers import AutoModel, AutoTokenizer
import gradio as gr
from langchain import PromptTemplate

# 新建 ChatGLMService 类
class ChatGLMService(LLM):
    max_token: int = 10000
    temperature: float = 0.1
    top_p = 0.9
    history = []
    tokenizer: object = None
    model: object = None

    def __init__(self):
        super().__init__()

    # 定义 llm 模型类型
    @property
    def _llm_type(self) -> str:
        return "ChatGLM"

    # 定义类调用的逻辑
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response, history = self.model.chat(
            self.tokenizer,
            prompt,
            history=self.history,
            max_length=self.max_token,
            temperature=self.temperature,
        )
        if stop is not None:
            response = enforce_stop_tokens(response, stop)  # 基于结束词将内容切断
        self.history = self.history + [history[-1]]  # 将问题和答案传到历史记录中
        return response

    # 加载 ChatGLM3-6B 模型
    def load_model(self, model_name_or_path: str = "chatglm3-6b"):
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path,
            trust_remote_code=True
        )
        self.model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).half().cuda()
        self.model = self.model.eval()

# 加载 ChatGLM3-6B 模型
work_dir = '/home/ma-user/work'
llm_model_name = 'chatglm3-6b'
llm_model_path = os.path.join(work_dir, llm_model_name)
chatLLM = ChatGLMService()
chatLLM.load_model(model_name_or_path=llm_model_path)

# 创建 Gradio 界面
def chat_with_glm(input_text):
    prompt_template = """\
请尽量简洁来回答用户的问题。
问题:
{question}
"""
    prompt = PromptTemplate(template=prompt_template, input_variables=["question"])
    message = prompt.format(question=input_text)
    response = chatLLM(message)
    return response

iface = gr.Interface(
    fn=chat_with_glm,
    inputs=gr.Textbox(show_label=False, placeholder="输入问题...", lines=3),
    outputs=gr.Textbox(show_label=False, placeholder="回答...", lines=3),
)

# 启动 Gradio 界面
iface.launch(share=True, inbrowser=True)


/home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:34<00:00,  4.94s/it]


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://313cf66a973c3570cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
